In [ ]:
import sys 
from io import BytesIO

sys.path.insert(0, "../")
import util
from test_framework.key import ECKey, ECPubKey
from test_framework.script import TapTree, TapLeaf, Node, CScript, OP_1, TaprootSignatureHash
from test_framework.address import program_to_witness
from test_framework.messages import CTransaction, COutPoint, CTxIn, CTxOut, CScriptWitness, CTxInWitness


# 2.3 TapTree

* Part 1 - Constructing a TapTree.
    * 1a - TapTree Descriptors.
    * 1b - TapTree Class.
    * 1c - Huffman Tree Constructor.
* Part 2 - Spending a Taproot output along the script path.


## Part 1: TapTree Construction.

### 1A - TapTree Descriptors

Descriptors are a high-level template language to describe individual or ranges of outputs. 

* Human readable & descriptive
* Imply standard script patterns
* Composable.

For taproot, we currently propose a taproot descriptor expression with the following pattern. 

![test](../images/TapTree0.jpg)

The tree structure is implied by the nested expression:
* **An internal node is represented by its children:**
    * Node expression: 
        * `[child_left, child_right]`
    * Node expressions are composable: 
        * `[child_left, [child_left', child_right']]`
* **A leaf node can be represented by its TapScript descriptor:**
    * For example, a pay-to-pubkey TapScript: 
        * `ts(pk(key))`
    * _More on TapScript descriptors in chapter 2.4._
    
*Note: The proposed taproot descriptor is not unique for a given taproot output. A given taproot output may have multiple correct taproot descriptor expressions. This is because we do not impose lexicographic ordering of the descriptor leafs and nodes as is done when computing the taproot.*

#### Example 1.1 - Constructing a Taptree from a descriptor.

![test](images/TapTree1.jpg)

In this example, we will construct the TapTree shown above from a descriptor literal.

* **Class: `TapTree`**
    * Construct from descriptor:
        * `TapTree.from_desc(descriptor_string)`
    * Serialize back to descriptor:
        * `TapTree.desc`

In [ ]:
# Generate internal keypairs
sec_internal = ECKey()
sec_internal.generate()
pk_internal = sec_internal.get_pubkey()

# Generaet keypairs for the tapleaf pay-to-pubkey scripts.
secA = ECKey()
secB = ECKey()
secC = ECKey()
secA.generate()
secB.generate()
secC.generate()
pkA = secA.get_pubkey()
pkB = secB.get_pubkey()
pkC = secC.get_pubkey()

# Construct Descriptor String.
ts_desc_A = 'ts(pk({}))'.format(pkA.get_bytes().hex())
ts_desc_B = 'ts(pk({}))'.format(pkB.get_bytes().hex())
ts_desc_C = 'ts(pk({}))'.format(pkC.get_bytes().hex())
tp_desc = 'tp({},[[{},{}],{}])'.format(pk_internal.get_bytes().hex(),
                                       ts_desc_A,
                                       ts_desc_B,
                                       ts_desc_C)

# Generate TapTree from descriptor.
taptree = TapTree()
taptree.from_desc(tp_desc)
print(taptree.desc)

# Compute Taproot Output:
taproot_script, tweak, control_map = taptree.construct()


### 1B - TapTree Construction

In addition to the TapTree class, we provide Node` and TapLeaf classes to capture internal and leaf nodes in a Taptree.

![test](../images/TapTree2.jpg)


* **Class `Node`:**
    * Node child members: 
        * `Node.left`, `Node.right`
        * (Left/Right child ordering does not affect taproot tweak derivation)

* **Class `TapLeaf`:**
    * Construct from descriptor string.
        * `TapLeaf.from_desc(descriptor_string)`
    
#### Example 1.2 - Constructing a Taptree with `Node ` & `TapLeaf`

In [ ]:
# Run example 1.1 first.

# Taproot
taptree = TapTree()
taptree.key = pk_internal

# TapLeafs
tapleaf_A = TapLeaf()
tapleaf_B = TapLeaf()
tapleaf_C = TapLeaf()
tapleaf_A.from_desc(ts_desc_A)
tapleaf_B.from_desc(ts_desc_B)
tapleaf_C.from_desc(ts_desc_C)

# TapTree
taproot = Node()
tapbranch_AB = Node()
tapbranch_AB.left = tapleaf_A
tapbranch_AB.right = tapleaf_B
taproot.left = tapbranch_AB
taproot.right = tapleaf_C
taptree.root = taproot

# Check if same construction as with taptree descriptor.
taproot_script2, tweak2, control_map2 = taptree.construct()

# Compare with example 1.1
assert(taproot_script2 == taproot_script)
print(taptree.desc)


### 1C - Huffman Constructor

A TapTree can also be constructed with a greedy algorithm based on the huffman encoding algorithm. 
Simply provide the huffman constructor method with the relative probability weights of each TapLeaf.

* **A higher weight** 
    * Indicates a higher likelihood of execution.
    * Means the script will be placed closer to the root if possible.
    * Results in a smaller inclusion proof and lower spending fees.

![test](../images/TapTree3.jpg)


#### Example 1.3 - Building a TapTree with the huffman constructor.
* We reconstruct the same taptree from examples 1.1 and 1.2 with the huffman constructor.

In [ ]:
# Run example 1.1 and 1.2 first.

taptree = TapTree()
taptree.key = pk_internal
taptree.huffman_constructor([(1, tapleaf_A), (1, tapleaf_B), (2, tapleaf_C)])
print(taptree.desc)

# Compare with example 1.1
taproot_script3, tweak3, control_map3 = taptree.construct()
assert(taproot_script3 == taproot_script)


## Part 2: Spending along the Script Path

A Taproot output is spent along the script path with the following witness pattern.

* Witness to spend TapScript_A:

    * `[Stack element(s) satisfying TapScript_A]`
    * `[TapScript_A]` 
    * `[Controlblock c]`


* Controlblock c contains:

    * `[Tapscript Version]` 
        * `0xfe & c[0]`
    * `[Internal Public Key]` 
        * y-coordinate: `0x01 & c[0]` 
        * x-coordinate: `c[1:33]`
    * `[Script Inclusion Proof]` 
        * `n x 32Bytes`


_Note: As of writing, the Taproot BIP has been updated to reflect 32B public keys in the controlblock. Following examples and exercises continue to use 33B public keys until the taproot Bitcoin Core branch has been updated._

![test](../images/TapTree4.jpg)


**Generating the Controlblock**

We use the the TapTree construct method to generate the taproot output, tweak and controlblocks for all TapScripts.

**`TapTree.construct()` returns the tuple:**
* `taproot_output_script`, `tweak`, `control_block_map`
* Control block map has key-value pairs: `tapscript` - `controlblock`
        

#### Programming Exercise 2.1 - Constructing a Taptree output.

In the following exercise, please construct the output and segwit address for the TapTree structure shown in section 1C with the huffman TapTree constructor. Please generate new keys for the internal key and pay-to-pubkey TapScripts.

We will cover TapScripts in more depth in the next chapter, but consider the following method to generate pay-to-pubkey TapScripts:
* `TapLeaf.construct_pk(public_key)`

In [ ]:
# Generate Keypairs for internal pubkey and pay-to-pubkey TapScripts.
sec_internal = ECKey()
sec_internal.generate()
pk_internal = sec_internal.get_pubkey()

secA = ECKey()
secB = ECKey()
secC = ECKey()
secD = ECKey()
secA.generate()
secB.generate()
secC.generate()
secD.generate()
pkA = secA.get_pubkey()
pkB = secB.get_pubkey()
pkC = secC.get_pubkey()
pkD = secD.get_pubkey()

# Construct Pay-toPubkey TapLeafs and Taptree.
TapLeafA = TapLeaf()
TapLeafB = TapLeaf()
TapLeafC = TapLeaf()
TapLeafD = TapLeaf()
TapLeafA.construct_pk(pkA)
TapLeafB.construct_pk(pkB)
TapLeafC.construct_pk(pkC)
TapLeafD.construct_pk(pkD)

taptree = TapTree()
taptree.key = pk_internal
taptree.huffman_constructor([(1,TapLeafA),(1, TapLeafB),(2, TapLeafC),(3,TapLeafD)])

# Generate taproot output and segwit address.
taproot_script, tweak, control_map = taptree.construct()
pk_taproot_bytes = bytes(taproot_script[2:])
segwit_address = program_to_witness(1, pk_taproot_bytes)
print(segwit_address)

#### Programming Exercise 2.2 - Spending a taproot output along a script path.

In this exercise, we will send funds to the previously generated address in exercise 2.1, and spend this output along the `TapScript0` path.
* TODO's in this exercise:
     * Create the appropriate sighash and signature.
     * Populate the `witness.stack` to spend `TapScriptA`
     

***Start TestNodes***

In [ ]:
# Start TestNodes.
test = util.TestWrapper()
test.setup(num_nodes=1)

***Generate Wallet Balance***

In [ ]:
# Generate Coins for Bitcoin Node Wallet.
test.nodes[0].generate(101)
balance = test.nodes[0].getbalance()
print(balance)

***Send funds from the wallet to the taproot output (Segwit Address).***

In [ ]:
# Send funds to taproot output.
txid = test.nodes[0].sendtoaddress(segwit_address, balance / 100000)
print("Funding tx:", txid)

# Deserialize wallet transaction.
tx = CTransaction()
tx_hex = test.nodes[0].getrawtransaction(txid)
tx.deserialize(BytesIO(bytes.fromhex(tx_hex)))
tx.rehash()

# Determine Output Index of Segwit V1 Output.
# (Wallet places change output at a random txout index.)
outputs = iter(tx.vout)
taproot_output = next(outputs)
index = 0

while (taproot_output.scriptPubKey != taproot_script):
    taproot_output = next(outputs)
    index += 1
output_value = taproot_output.nValue

***Spend taproot output with along TapScript A.***

In [ ]:
tx_script_spend = CTransaction()
tx_script_spend.nVersion = 1
tx_script_spend.nLockTime = 0
outpoint = COutPoint(tx.sha256, index)
tx_script_spend_input = CTxIn(outpoint = outpoint)
tx_script_spend.vin = [tx_script_spend_input]

# Generate new Bitcoin Core wallet address to send funds back to.
dest_addr = test.nodes[0].getnewaddress(address_type="bech32")
scriptpubkey = bytes.fromhex(test.nodes[0].getaddressinfo(dest_addr)['scriptPubKey'])

# Determine minimum fee required for mempool acceptance.
min_fee = int(test.nodes[0].getmempoolinfo()['mempoolminfee'] * 100000000)

# Complete output which returns funds to Bitcoin Core wallet.
dest_output= CTxOut(nValue=output_value-min_fee, scriptPubKey=scriptpubkey)
tx_script_spend.vout = [dest_output]

# Sign transaction with tweaked private key.
# TODO: sig =
hash_types = [0,1,2,3,0x81,0x82,0x83]
sighash = TaprootSignatureHash(tx_script_spend, 
                               [taproot_output], 
                               hash_types[0], 
                               input_index = 0, 
                               scriptpath = True, 
                               tapscript = TapLeafA.script)
sig = secA.sign_schnorr(sighash)

# TODO: Populate transaction witness.
witness = CScriptWitness()
witness.stack = [sig, TapLeafA.script, control_map[TapLeafA.script]]
witness_in = CTxInWitness()
witness_in.scriptWitness = witness
tx_script_spend.wit.vtxinwit.append(witness_in)

# Serialize Schnorr transaction for broadcast.
tx_script_spend_str = tx_script_spend.serialize().hex()

# Test mempool acceptance.
print(test.nodes[0].testmempoolaccept([tx_script_spend_str]))

***Shutdown TestNodes***

In [ ]:
# Shutdown TestNodes.
test.shutdown()